In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.
        builder.
        appName("curso_pyspark").
        getOrCreate()
)

spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/29 20:25:26 WARN Utils: Your hostname, Matheuss-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.135 instead (on interface en0)
25/07/29 20:25:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/29 20:25:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Básico de Pyspark

## Dataframes

<!-- ###
### create table (
###    nome VARCHAR(100) not null,
###    sobrenome VARCHAR(100) not null,
###    idade INT not null
###)

#  nome sobrenome idade
# (N1      N2       I1) -->

create table ( <br />
   nome VARCHAR(100) not null, <br />
   sobrenome VARCHAR(100) not null, <br />
   idade INT not null <br />
) <br />
<br /><br />
nome sobrenome idade
(N1      N2       I1)

In [2]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType

data = [
    ##  C1          C2        C3
    ("Matheus", "Cantarutti", 31),
    ("Ana", "Cláudia", 18),
    ("Brunno", "Oliveira", 25)
]

schema = StructType([
    StructField("Nome", StringType(), True),
    StructField("Sobre_Nome", StringType(), True),
    StructField("Idade", IntegerType(), True)
])

df = spark.createDataFrame(data, schema)
df.printSchema()

df.show()

root
 |-- Nome: string (nullable = true)
 |-- Sobre_Nome: string (nullable = true)
 |-- Idade: integer (nullable = true)



+-------+----------+-----+
|   Nome|Sobre_Nome|Idade|
+-------+----------+-----+
|Matheus|Cantarutti|   31|
|    Ana|   Cláudia|   18|
| Brunno|  Oliveira|   25|
+-------+----------+-----+



In [3]:
df.createOrReplaceTempView("pessoas")

In [4]:
### SQL --> Pyspark

spark.sql( 
'''    
    select
        *
    from pessoas
    where Idade < 20
'''
).show()

+----+----------+-----+
|Nome|Sobre_Nome|Idade|
+----+----------+-----+
| Ana|   Cláudia|   18|
+----+----------+-----+



In [5]:
from pyspark.sql import functions as F

# df.filter('Idade < 20').show()
df.filter(
    F.col('Idade') < 20
).show()

+----+----------+-----+
|Nome|Sobre_Nome|Idade|
+----+----------+-----+
| Ana|   Cláudia|   18|
+----+----------+-----+



## Tipo de Dados

- TIPO TEXTO/STRING >> abrangendo apenas as funções que tratam texto
- TIPO DATA (DATA ESTÁ COM O TIPO DE STRING) >> Converter o seu texto para Data

- FLOAT/DECIMAL e INTERGER

In [12]:
spark.sql('''
    select
        *,
        cast(Idade * 5 as string) as Idade_2
    from pessoas
''').printSchema()

root
 |-- Nome: string (nullable = true)
 |-- Sobre_Nome: string (nullable = true)
 |-- Idade: integer (nullable = true)
 |-- Idade_2: string (nullable = true)



In [17]:
(   
    # nome da coluna, expressões/funcao
    df.withColumn('Idade_2', F.col('Idade') * 5)
      .withColumn('data', F.lit('2025-01-01')) # current date
      .withColumn('data2', F.to_date(F.col('data'), 'yyyy-MM-dd'))
      .withColumn('Idade_3', F.expr('cast(Idade * 5 as string) as Idade_3'))

).show()

+-------+----------+-----+-------+----------+----------+-------+
|   Nome|Sobre_Nome|Idade|Idade_2|      data|     data2|Idade_3|
+-------+----------+-----+-------+----------+----------+-------+
|Matheus|Cantarutti|   31|    155|2025-01-01|2025-01-01|    155|
|    Ana|   Cláudia|   18|     90|2025-01-01|2025-01-01|     90|
| Brunno|  Oliveira|   25|    125|2025-01-01|2025-01-01|    125|
+-------+----------+-----+-------+----------+----------+-------+



## Cardinalidade

- Aula teórica explicativa sobre cardinalidade